# Building Text Summarization Engine

One of the popular algorithm for text summarization is text rank. Text is an extractive and unsupervised techniques for the text summarization. Text rank is inspired by the page rank algorithm. page rank is popularly used in the search engine development like google.  Let's see how page rank works. For Example, we have 5 pages with some hyperlinking between them. The hyperlink is a way to navigate between two pages.  the connectivity of one page is defined as per the number of hyperlinks present between them. for example, let's say we have following hyperlinking in 5 pages

The rank of these pages can be defined on the basis of a number of links present in between them using Pagerank algorithm.

The connectivity between 5 pages can be represented with probabilities and this probability can be easily represented with a matrix of 5*5 as shown below with all the probabilities. 

Write about matrix generation. The page which is having no connectivity is called a dangling page. here --- is the dangling page. 

The entire process of the Text rank is can be summarized in the following steps.these step is very similar to the Page rank algorithm. 

- In place of web pages, we use text in the Text rank
- The hyperlink between two pages as the connectivity measure is replaced with the sentence similarity in the text rank algorithm.
- The matrix is constructed with a similarity between all the sentences.


In the single document summarization, the document is tokenized.
In the multiple documents, the first step would concatenate all the documents and then tokenize the text.
Apply distance measure on the sentence after applying embeddings to the tokens. Using this a similarity matrix is calculated. 
A weighted graph is constructed. 
page rank is applied to the constructed graph to form the sentence ranking
Based on the sentence ranking summary s constructed


# Importing Requirements

In [14]:
import numpy as np
import pandas as pd
import nltk
import re
import chakin
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
import networkx as nx

Constructing a dataframe by reading our created dadaset.

In [5]:
dataframe = pd.read_csv("data/text_summarization_dataset.tsv","\t")

The constructed dataframe looks like as given below, It has three columns id, url and text. The url is given to comply with the commercial use of wikipedia content. The text is the only one colum we will be using here onward. We have seven short paragrah about galaxies taken from wiikipedia. This can be considered as multiple document and our task is to findout/extract few sentences that represent the crux of the all seven documents.

In [6]:
dataframe 

,id,url,text
0,1,https://en.wikipedia.org/wiki/Galaxy#Interacting,Interactions between galaxies are relatively f...
1,2,https://en.wikipedia.org/wiki/Galaxy#Interacting,Stars are created within galaxies from a reser...
2,3,https://en.wikipedia.org/wiki/Galaxy#Interacting,A portion of the observable galaxies are class...
3,4,https://en.wikipedia.org/wiki/Dark_galaxy,A dark galaxy is a hypothesized galaxy with no...
4,5,https://en.wikipedia.org/wiki/Dark_galaxy,Scientists do not have much explanation for so...
5,6,https://en.wikipedia.org/wiki/Dark_matter,Dark matter is a form of matter that is though...
6,7,https://en.wikipedia.org/wiki/Dark_matter,The primary evidence for dark matter is that c...
7,8,https://en.wikipedia.org/wiki/Dark_matter,Because dark matter has not yet been observed ...
8,9,https://en.wikipedia.org/wiki/Dark_matter,The arms of spiral galaxies rotate around the ...
9,10,https://en.wikipedia.org/wiki/Dark_matter,One of the consequences of general relativity ...


In [8]:
dataframe['text'][0]

'Interactions between galaxies are relatively frequent, and they can play an important role in galactic evolution. Near misses between galaxies result in warping distortions due to tidal interactions, and may cause some exchange of gas and dust.[83][84] Collisions occur when two galaxies pass directly through each other and have sufficient relative momentum not to merge. The stars of interacting galaxies will usually not collide, but the gas and dust within the two forms will interact, sometimes triggering star formation. A collision can severely distort the shape of the galaxies, forming bars, rings or tail-like structures.[83][84]. At the extreme of interactions are galactic mergers. In this case the relative momentum of the two galaxies is insufficient to allow the galaxies to pass through each other. Instead, they gradually merge to form a single, larger galaxy. Mergers can result in significant changes to morphology, as compared to the original galaxies. If one of the merging gala

Tokenizing paragraph in to sentences and accumlating all sentecnes in to one list

In [12]:
from nltk.tokenize import sent_tokenize
sentences = []
for each_paragraph in dataframe['text']:
    for each_sent in sent_tokenize(each_paragraph):
        sentences.append(each_sent)

Downloading Embeddings : For this experimentation, I will be using GloVe vector of dimension 100 trained on "Wikipedia+Gigaword 5 (6B)" dataset. I will be using chakin to download GloVe word vectors. Once the vector is downloaded the vocabulary for our train and test split is mapped to GloVe vector by using below given snippet. Remember this method because we will be using this shortcut at many places in this chapter.

In [ ]:
embed_exists = os.path.isfile('../embeddings/glove.6B.zip')
if not embed_exists:
    print("Downloading Glove embeddings, if not downloaded properly, then delete the `embeddings/glove.6B.zip")
    chakin.search(lang='English')
    chakin.download(number=16, save_dir='../embeddings')
    zip_ref = zipfile.ZipFile("../embeddings/glove.6B.zip", 'r')
    zip_ref.extractall("../embeddings/")
    zip_ref.close()

Vectorization : We will be using glove 100-dimensional vectors to convert our word in to vectors. The below given function will load all the vectors to word_embeddings dictionary where key will be word and value will be its 100 dimensional vector. Though this is not the best way but for loading embeddings but for this experiemtn its the quickest way to get the things done.

In [15]:
# Extract word vectors
word_embeddings = {}
f = open('../embeddings/glove.6B/glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    vector = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = vector
f.close()

Some preprocessing steps.

In [16]:
# remove punctuations, numbers and special characters
clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")
# make alphabets lowercase
clean_sentences = [s.lower() for s in clean_sentences]

In [17]:
# function to remove stopwords
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new

# remove stopwords from the sentences
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

Sentence Representation : Next step is to find average of all the word embeddings in the sentence, in this way each sentence will be represrented by centroid of all the word

In [18]:
sentence_vectors = []
for each_sent in clean_sentences:
    if len(each_sent) != 0:
        v = sum([word_embeddings.get(word, np.zeros((100,))) for word in each_sent.split()])/(len(each_sent.split())+0.001)
    else:
        v = np.zeros((100,))
    sentence_vectors.append(v)

The resultant shape of the sentence will be equal to number of sentence each will be of 100-dimensions.

In [19]:
np.array(sentence_vectors).shape

(85, 100)

Initializing similarity matrix, the similarity matrix will be a square matrix of width and height equal to number sentences.

In [20]:
similarity_mat = np.zeros([len(sentences), len(sentences)])

Finding distance betwenn all sentence to all sentences by getting cosine similarity between sentence. Any other distance measure cna be used.

In [22]:
for i in range(len(sentences)):
    for j in range(len(sentences)):
        if i != j:
            similarity_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]

Using `NetworkX` as the library to apply page rank on our similarity matrix.

In [24]:
nx_graph = nx.from_numpy_array(similarity_mat)
scores = nx.pagerank(nx_graph)

The score are generated for each sentence and sentences are ranked according to score. 

In [25]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)

Extract top 10 sentences as the summary

In [26]:
for i in range(10):
  print(i,">",ranked_sentences[i][1])

0 > The primary evidence for dark matter is that calculations show that many galaxies would fly apart instead of rotating, or would not have formed or move as they do, if they did not contain a large amount of unseen matter.
1 > He goes on to say, "In our current theory of galaxy formation, we believe that big galaxies form from the merger of smaller galaxies.
2 > The stars of interacting galaxies will usually not collide, but the gas and dust within the two forms will interact, sometimes triggering star formation.
3 > Little is known about dark galaxies, and some scientists believe a dark galaxy is actually a newly forming galaxy.
4 > A significant portion of the total energy output from the galaxy is emitted by the active galactic nucleus, instead of the stars, dust and interstellar medium of the galaxy.The standard model for an active galactic nucleus is based upon an accretion disc that forms around a supermassive black hole (SMBH) at the core region of the galaxy.
5 > [49] Instead